# Hecho de servicio por fases

In [1]:
import datetime
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Definir las fases del servicio
fases = [
    'Iniciado',
    'Con mensajero asignado',
    'Recogido en origen',
    'Entregado en destino',
    'Cerrado'
]

# Añadir conexión a la base de datos

In [ ]:
with open('../config.yml', 'r') as f: 
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']


url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")


ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Consultar datos de servicios y fases
query_servicios = "SELECT id_servicio, fecha_inicio, fecha_fin, estado FROM servicios"
query_fases = "SELECT id_fase, id_servicio, fase FROM fases"

# Leer los datos en DataFrames de pandas
df_servicios = pd.read_sql(query_servicios, ryf_conn)
df_fases = pd.read_sql(query_fases, ryf_conn

In [ ]:
#Calculo de las fases

tiempos_espera = pd.DataFrame(columns=['fase', 'tiempo_espera'])

# Calcular tiempos de espera por fase
for i in range(len(fases) - 1):
    fase_actual = fases[i]
    fase_siguiente = fases[i + 1]
    
    # Filtrar los servicios en la fase actual
    df_fase_actual = tabla_hechos_final[tabla_hechos_final['fase'] == fase_actual]

    # Calcular el tiempo de espera
    if not df_fase_actual.empty:
        tiempo_espera = (df_fase_actual['fecha_inicio'].shift(-1) - df_fase_actual['fecha_fin']).dt.days
        tiempo_espera = tiempo_espera[timer_espera > 0]  # Considerar solo tiempos positivos

        # Agregar el resultado al DataFrame de tiempos de espera
        tiempos_espera = tiempos_espera.append({'fase': fase_actual, 'tiempo_espera': tiempo_espera.mean()}, ignore_index=True)

# Agregar la última fase (Cerrado) con el tiempo de fin
df_cerrado = tabla_hechos_final[tabla_hechos_final['fase'] == 'Cerrado']
tiempo_cerrado = (df_cerrado['fecha_fin'] - df_cerrado['fecha_inicio']).dt.days.mean()
tiempos_espera = tiempos_espera.append({'fase': 'Cerrado', 'tiempo_espera': tiempo_cerrado}, ignore_index=True)

Extracción y procesamiento

In [ ]:
df_servicios['fecha_inicio'] = pd.to_datetime(df_servicios['fecha_inicio'])
df_servicios['fecha_fin'] = pd.to_datetime(df_servicios['fecha_fin'])

df_servicios['duracion'] = (df_servicios['fecha_fin'] - df_servicios['fecha_inicio']).dt.days

# Unir los DataFrames para crear la tabla de hechos
tabla_hechos = df_fases.merge(df_servicios, on='id_servicio', how='inner')

tabla_hechos_final = tabla_hechos[['id_servicio', 'id_fase', 'fecha_inicio', 'fecha_fin', 'duracion', 'estado']]
tabla_hechos_final

guardar las tablas

In [ ]:
tabla_hechos_final.to_sql('tabla_hechos_servicios', etl_conn, if_exists='replace', index=False)

# O guardar como archivo CSV
tabla_hechos_final.to_csv('tabla_hechos_servicios.csv', index=False)

8) Mostrar los tiempos de espera por cada fase del servicio: Iniciado, Con mensajero asignado, recogido en origen, Entregado en Destino, Cerrado. En que fase del servicio hay más demoras?

In [ ]:
print(tiempos_espera)

# Encontrar la fase con más demoras
fase_max_demora = tiempos_espera.loc[tiempos_espera['tiempo_espera'].idxmax()]
print(f"La fase con más demoras es: {fase_max_demora['fase']} con un tiempo de espera promedio de {fase_max_demora['tiempo_espera']} días.")